# Segmenting and Clustering Neighborhoods in Paris

## My project description

<div style="margin-top: 20px">

<font size = 2>
In my Capstone project I want to study the different tourist places by neighborhood  of Paris, the city of lights. Study the different small activities around each tourist place such as restaurants, libraries, etc. Also study the correlation between the population by neighborhood and the number of restaurants and their type.
 
</font>
</div>

### Data source

To do this project, I retrieve the data from the opendata site https://opendata.paris.fr/page/home/, by doing web scraping with beautifulsoup, or dowloading the data in a csv file or using the restful API.

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install lxml
import lxml

     |████████████████████████████████| 112kB 9.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.8MB 8.3MB/s eta 0:00:01


In [2]:
!wget  https://opendata.paris.fr/explore/dataset/quartier_paris/download?format=csv -O paris_quartiers.csv

--2020-02-17 16:18:59--  https://opendata.paris.fr/explore/dataset/quartier_paris/download?format=csv
Resolving opendata.paris.fr (opendata.paris.fr)... 34.248.20.69, 34.249.199.226
Connecting to opendata.paris.fr (opendata.paris.fr)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘paris_quartiers.csv’

paris_quartiers.csv     [     <=>            ] 340.17K   236KB/s    in 1.4s    

2020-02-17 16:19:02 (236 KB/s) - ‘paris_quartiers.csv’ saved [348339]



In [3]:
df_data = pd.read_csv('paris_quartiers.csv',sep=';')
df_data.head()

,n_sq_qu,c_qu,c_quinsee,l_qu,c_ar,n_sq_ar,perimetre,surface,geom_x_y,geom
0,750000021,21,7510601,Monnaie,6,750000006,2391.122818,2.933606e+05,"48.8543844036,2.34003537113","{""type"": ""Polygon"", ""coordinates"": [[[2.343168..."
1,750000022,22,7510602,Odéon,6,750000006,3516.314464,7.161484e+05,"48.8478006293,2.33633882759","{""type"": ""Polygon"", ""coordinates"": [[[2.336988..."
2,750000029,29,7510801,Champs-Elysées,8,750000008,5480.652870,1.140254e+06,"48.8670744922,2.30865168468","{""type"": ""Polygon"", ""coordinates"": [[[2.306463..."
3,750000051,51,7511303,Maison-Blanche,13,750000013,6814.373288,2.231506e+06,"48.8231278057,2.35243314954","{""type"": ""Polygon"", ""coordinates"": [[[2.349115..."
4,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,"48.8337336761,2.34767304607","{""type"": ""Polygon"", ""coordinates"": [[[2.351662..."


## Data analylis and cleaning data for using it

In [4]:

paris_data = df_data.loc[:,['c_ar','geom_x_y','c_quinsee','l_qu']]
paris_data['c_quinsee'] = paris_data['c_quinsee'].astype(str)
paris_data['code'] = paris_data['c_quinsee'].str[0:5].astype(str)
paris_data['Latitude'] = paris_data['geom_x_y'].str.extract('(.*)\,')
paris_data['Longitude'] = paris_data['geom_x_y'].str.extract(',(.*)')
paris_data = paris_data.drop(['geom_x_y','c_quinsee'],axis=1)
paris_data.columns= ['Borough', 'Neighbourhood', 'Code', 'Latitude','Longitude']
#paris_data = paris_data.reset_index()

paris_data['Borough'] = paris_data['Borough'].astype(str)
paris_data['Code'] = paris_data['Code'].astype(str)
paris_data['Latitude'] = paris_data['Latitude'].astype(float)
paris_data['Longitude'] = paris_data['Longitude'].astype(float)

paris_data.head()

,Borough,Neighbourhood,Code,Latitude,Longitude
0,6,Monnaie,75106,48.854384,2.340035
1,6,Odéon,75106,48.847801,2.336339
2,8,Champs-Elysées,75108,48.867074,2.308652
3,13,Maison-Blanche,75113,48.823128,2.352433
4,13,Croulebarbe,75113,48.833734,2.347673


## Load data of paris's borough population

In [5]:
page_url  ='https://fr.wikipedia.org/wiki/Arrondissements_de_Paris'

response = requests.get(page_url)

if response is not None:
    soup = BeautifulSoup(response.text, 'html.parser')


In [6]:
#soup.prettify()
table = soup.find("table", {"class":"wikitable sortable alternance jquery-tablesorter"})
print(table)

None


# Displaying Paris's Map

In [7]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


In [8]:
# create map of Paris using latitude and longitude values
longitude = 2.3488

latitude = 48.85341
 
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, codepost, neighborhood, borough in zip(paris_data['Latitude'], paris_data['Longitude'], paris_data['Code'], paris_data['Neighbourhood'], paris_data['Borough']):
    label = '{},{}, {} ème'.format(codepost, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris) 

map_paris

# Using Foursquare data to load Paris's venues 

Set credentials

In [9]:
CLIENT_ID = 'AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM' # your Foursquare ID
CLIENT_SECRET = '0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0' # your Foursquare Secret
VERSION = '20200210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAXS11JUTRNPPC4LYAOQYLTGYUAH13CIHKHXHJEQJY4ERUDM
CLIENT_SECRET:0QDSFHVBM3Z1DPVSSWXMCESYJNHNPPJMRT4OJUGZV3IFNDC0


Infos for the first dataframe's rows

In [10]:
neighborhood_latitude = paris_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = paris_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = paris_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Monnaie are 48.8543844036, 2.34003537113.


Build Foursquare URL with geo coordinates

In [11]:

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Load data from Foursquare

In [12]:
results = requests.get(url).json()
#results

Display result keys in json file

In [13]:
results['response']['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

Function that extracts the category of the venue

In [14]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Normalizing json of venues

In [15]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Prescription Cocktail Club,Cocktail Bar,48.854921,2.337768
1,Chez Nous,Wine Bar,48.855823,2.340271
2,Maison Sauvage,French Restaurant,48.853744,2.337993
3,Chez Le Libanais,Lebanese Restaurant,48.853285,2.341673
4,Ze Kitchen Galerie,Restaurant,48.855101,2.341297


List of Categories

In [16]:
nearby_venues['categories'].unique()

array(['Cocktail Bar', 'Wine Bar', 'French Restaurant',
       'Lebanese Restaurant', 'Restaurant', 'Hotel', 'Breton Restaurant',
       'Bookstore', 'Ice Cream Shop', 'Sandwich Place',
       'Japanese Restaurant', 'Modern European Restaurant',
       'Chocolate Shop', 'Tapas Restaurant', 'Seafood Restaurant',
       'Italian Restaurant', 'Tea Room', 'Ramen Restaurant',
       'Coffee Shop', 'Plaza', 'Museum', 'Café', 'Liquor Store',
       'Beer Bar', 'Argentinian Restaurant', 'Creperie', 'Burger Joint',
       'Bistro', 'Gourmet Shop', 'Wine Shop', 'Pub', 'Spanish Restaurant',
       'Mexican Restaurant', 'Pedestrian Plaza', 'Park', 'Fountain',
       'Sports Bar', 'Bar', 'Electronics Store', 'Asian Restaurant',
       'Greek Restaurant', 'Indie Movie Theater', 'Brasserie',
       'Pastry Shop', 'Cajun / Creole Restaurant', 'Clothing Store',
       'Pizza Place', 'Dessert Shop', 'Breakfast Spot', 'Candy Store',
       'Cosmetics Shop'], dtype=object)

Number of venues from Foursquare

In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Method loading all venues for each neiborhoud

In [21]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Dsiplay data from Foursquare

In [22]:
paris_venues = getNearbyVenues(names=paris_data['Neighbourhood'],
                                   latitudes=paris_data['Latitude'],
                                   longitudes=paris_data['Longitude']
                                  )

paris_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Monnaie,48.854384,2.340035,Prescription Cocktail Club,48.854921,2.337768,Cocktail Bar
1,Monnaie,48.854384,2.340035,Chez Nous,48.855823,2.340271,Wine Bar
2,Monnaie,48.854384,2.340035,Maison Sauvage,48.853744,2.337993,French Restaurant
3,Monnaie,48.854384,2.340035,Chez Le Libanais,48.853285,2.341673,Lebanese Restaurant
4,Monnaie,48.854384,2.340035,Ze Kitchen Galerie,48.855101,2.341297,Restaurant


Grouping by neighborhood

In [23]:
paris_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Amérique,12,12,12,12,12,12
Archives,100,100,100,100,100,100
Arsenal,71,71,71,71,71,71
Arts-et-Métiers,100,100,100,100,100,100
Auteuil,15,15,15,15,15,15
Batignolles,98,98,98,98,98,98
Bel-Air,5,5,5,5,5,5
Belleville,44,44,44,44,44,44
Bercy,65,65,65,65,65,65


## One Hot encoding by "Venue Category"

In [24]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
paris_onehot['Neighbourhood'] = paris_venues['Neighbourhood'] 


# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse

In [26]:
paris_onehot.shape

(5458, 304)

In [28]:
paris_grouped = paris_onehot.groupby('Neighbourhood').mean().reset_index()
paris_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse

In [39]:
paris_grouped.shape

(80, 304)

Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in paris_grouped['Neighbourhood']:
    print('----------',hood,'----------')
    temp = paris_grouped[paris_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns=['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---------- Amérique ----------
               venue  freq
0  French Restaurant  0.17
1        Supermarket  0.17
2               Park  0.08
3               Café  0.08
4    Bed & Breakfast  0.08


---------- Archives ----------
               venue  freq
0  French Restaurant  0.08
1     Clothing Store  0.08
2        Coffee Shop  0.06
3       Burger Joint  0.04
4        Pastry Shop  0.04


---------- Arsenal ----------
               venue  freq
0  French Restaurant  0.13
1              Hotel  0.11
2              Plaza  0.04
3          Gastropub  0.04
4               Park  0.04


---------- Arts-et-Métiers ----------
                venue  freq
0   French Restaurant  0.10
1               Hotel  0.07
2  Italian Restaurant  0.05
3        Cocktail Bar  0.05
4         Coffee Shop  0.04


---------- Auteuil ----------
               venue  freq
0       Tennis Court  0.33
1            Stadium  0.13
2         Racecourse  0.07
3             Museum  0.07
4  French Restaurant  0.07


---------- Bat

### Let's put that into a pandas dataframe